# Assign Map Colors

For a new map after redistricing, try to make the colors match when switching between map versions.

Goal: If 1C01 is light blue on the 2012 map, then the same area should also be light blue on the 2022 map. The district number might have changed, though! 

In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
districts = pd.read_csv('../data/districts.csv')
overlap = pd.read_csv('overlap_forwards.csv')

In [3]:
map_2022 = gpd.read_file('Single_Member_District_from_2023.geojson')

In [4]:
map_2022['smd_name'] = map_2022['SMD_ID']
map_2022['smd_id'] = 'smd_' + map_2022['SMD_ID']
map_2022.rename(columns={'ANC_ID': 'anc_id'}, inplace=True)

In [5]:
# We only need these columns going forward
map_columns = ['smd_id', 'smd_name', 'anc_id', 'geometry']
map_2022 = map_2022[map_columns].copy()

In [6]:
overlap_top_match = overlap[overlap.district_rank == 1].copy()

In [7]:
# Left join the 2022 map to the 2012 district overlap
district_overlap = pd.merge(
    districts[['smd_id', 'map_color_id']], overlap_top_match, how='left', left_on='smd_id', right_on='smd_id_2012'
)

# Drop this duplicate column to avoid confusion and collision
district_overlap.drop(columns=['smd_id'], inplace=True)

map_2022_colors = map_2022.merge(
    district_overlap, how='left', left_on='smd_id', right_on='smd_id_2022'
)

In [8]:
map_2022_colors['map_color_id'] = map_2022_colors['map_color_id'].fillna(13)

In [9]:
# Make sure this column in an integer. Because it was previously NULL, pandas set it a decimal
map_2022_colors['map_color_id'] = map_2022_colors['map_color_id'].astype('int64')

In [10]:
map_2022_colors['qgis_label'] = '2022_' + map_2022_colors['smd_name']

In [11]:
map_2022_colors.head()

,smd_id,smd_name,anc_id,geometry,map_color_id,smd_id_2022,district_area_2022,smd_id_2012,district_area_2012,overlap_area,overlap_perc,district_rank,qgis_label
0,smd_1C08,1C08,1C,"POLYGON ((-77.03649 38.92274, -77.03663 38.922...",8,smd_1C08,128342.785633,smd_1C08,139664.389254,92014.851455,0.658828,1.0,2022_1C08
1,smd_1C07,1C07,1C,"POLYGON ((-77.03918 38.92494, -77.03921 38.924...",7,smd_1C07,129160.040163,smd_1C07,203187.796445,111510.190211,0.548804,1.0,2022_1C07
2,smd_1C03,1C03,1C,"POLYGON ((-77.04269 38.92267, -77.04282 38.922...",3,smd_1C03,119796.170816,smd_1C03,119796.342840,119796.031763,0.999997,1.0,2022_1C03
3,smd_1C02,1C02,1C,"POLYGON ((-77.04478 38.91906, -77.04484 38.919...",13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022_1C02
4,smd_1C01,1C01,1C,"POLYGON ((-77.04168 38.91875, -77.04180 38.918...",1,smd_1C01,135254.012713,smd_1C01,154959.758434,127584.174939,0.823337,1.0,2022_1C01


In [12]:
map_2022_colors.groupby('map_color_id').size()

map_color_id
1     29
2     26
3     27
4     24
5     23
6     22
7     23
8     25
9     24
10    26
11    24
12    23
13    82
dtype: int64

In [13]:
map_2022_colors[map_columns + ['map_color_id', 'qgis_label']].to_file(
    'to-mapbox-2022-smd-data-initial.geojson', driver='GeoJSON')

In [14]:
# no_geo = [c for c in map_2022_overlap.columns if c != 'geometry']
# map_2022_overlap[no_geo].sort_values(by='smd_id').to_clipboard()